In [1]:
import pandas as pd

import psycopg2 as pg2
con = pg2.connect(host='this_postgres', user='postgres',database='postgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

Check running processes

In [2]:
sql = '''
SELECT pid , query, *
FROM pg_stat_activity pgsa
WHERE pid <> pg_backend_pid()
'''

In [3]:
df = select(sql)

In [4]:
df

pid                           query    datid   datname  pid  usesysid  \
0   52                                      NaN      None   52      10.0   
1   50                                      NaN      None   50       NaN   
2   55                          COMMIT  13044.0  postgres   55      10.0   
3   54                        ROLLBACK  13044.0  postgres   54      10.0   
4   59  select * from sales t limit 10  13044.0  postgres   59      10.0   
5   48                                      NaN      None   48       NaN   
6   47                                      NaN      None   47       NaN   
7   49                                      NaN      None   49       NaN   

    usename application_name client_addr client_hostname  ...  \
0  postgres                         None            None  ...   
1      None                         None            None  ...   
2  postgres                   172.19.0.2            None  ...   
3  postgres                   172.19.0.2            None  ...   
4  postgres                   172.19.0.2            None  ...   
5      None                         None            None  ...   
6      None                         None            None  ...   
7      None                         None            None  ...   

                        xact_start                      query_start  \
0                              NaT                              NaT   
1                              NaT                              NaT   
2                              NaT 2019-12-29 06:48:19.708090+00:00   
3                              NaT 2019-12-29 06:46:08.270005+00:00   
4 2019-12-29 06:48:41.352746+00:00 2019-12-29 06:48:44.075011+00:00   
5                              NaT                              NaT   
6                              NaT                              NaT   
7                              NaT                              NaT   

                      state_change wait_event_type           wait_event  \
0                              NaT        Activity  LogicalLauncherMain   
1                              NaT        Activity       AutoVacuumMain   
2 2019-12-29 06:48:19.710595+00:00          Client           ClientRead   
3 2019-12-29 06:46:08.270066+00:00          Client           ClientRead   
4 2019-12-29 06:48:44.075685+00:00          Client           ClientRead   
5                              NaT        Activity         BgWriterMain   
6                              NaT            None                 None   
7                              NaT        Activity        WalWriterMain   

                 state backend_xid backend_xmin  \
0                 None        None         None   
1                 None        None         None   
2                 idle        None         None   
3                 idle        None         None   
4  idle in transaction        None         None   
5                 None        None         None   
6                 None        None         None   
7                 None        None         None   

                            query         backend_type  
0                                    background worker  
1                                  autovacuum launcher  
2                          COMMIT       client backend  
3                        ROLLBACK       client backend  
4  select * from sales t limit 10       client backend  
5                                    background writer  
6                                         checkpointer  
7                                            walwriter  

[8 rows x 22 columns]

Kill all

In [5]:
sql = '''
SELECT 
    pg_terminate_backend(pid) 
FROM 
    pg_stat_activity 
WHERE 
    -- don't kill my own connection!
    pid <> pg_backend_pid()

'''

In [6]:
cur.execute(sql)